In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/capstone/Train_Data (1).csv")

In [ ]:
train_data.head(10)

,pc,ld,m0,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,m14,ma,pred
0,y,83.71,63.0,7.2,52.5,14.0232,130.8300,12.4280,188.8425,8.9520,201.1905,9.2896,141.9075,16.0968,150.3390,12.4880,173.1240,m78,0
1,y,108.94,31.5,12.8,84.0,13.2840,128.8350,13.5256,183.0990,8.8520,207.2385,8.4704,154.7805,13.3304,101.0205,12.5096,131.4075,m78,0
2,E,169.65,0.0,5.6,73.5,14.5472,128.9295,13.6424,174.4680,8.9800,190.3125,11.3056,156.7650,NaN,122.5350,11.7136,176.8200,m76,1
3,x,122.42,31.5,7.2,63.0,15.0152,119.8575,12.3344,186.8580,10.7208,193.8195,10.6096,175.7490,NaN,124.8030,13.8424,168.2625,m55,0
4,E,125.43,94.5,7.2,42.0,14.4176,135.4290,14.5824,187.8135,9.3088,203.1540,9.8280,172.7040,14.4720,120.2145,NaN,150.1185,m76,0
5,y,160.61,84.0,10.4,73.5,13.4656,121.3485,14.2832,188.7480,10.6888,195.7095,9.8808,167.1810,15.5344,103.9920,13.8584,169.7850,m78,0
6,w,112.77,73.5,4.0,73.5,12.7136,110.5125,NaN,187.6350,9.1656,209.5905,7.2080,160.9230,16.4504,104.1810,13.3848,190.5015,m78,1
7,z,136.55,73.5,9.6,94.5,14.1792,121.4430,13.8368,189.3465,8.8304,211.2705,9.3368,165.4485,15.8424,124.3725,12.5552,158.0775,m75,0
8,z,101.09,94.5,8.8,252.0,13.5632,109.3785,13.5984,185.8710,8.8368,188.7165,8.5168,163.4010,15.9320,113.0115,13.4072,150.6120,m75,1
9,E,110.30,31.5,5.6,84.0,15.3704,135.1560,12.8016,182.2065,8.1688,212.8665,9.6952,146.9475,15.5800,151.3155,11.9408,208.0575,m76,0


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22584 entries, 0 to 22583
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pc      22584 non-null  object 
 1   ld      22362 non-null  float64
 2   m0      22584 non-null  float64
 3   m1      22584 non-null  float64
 4   m2      22584 non-null  float64
 5   m3      22270 non-null  float64
 6   m4      22117 non-null  float64
 7   m5      22008 non-null  float64
 8   m6      21909 non-null  float64
 9   m7      21796 non-null  float64
 10  m8      21684 non-null  float64
 11  m9      21537 non-null  float64
 12  m10     21484 non-null  float64
 13  m11     21357 non-null  float64
 14  m12     21203 non-null  float64
 15  m13     21085 non-null  float64
 16  m14     20995 non-null  float64
 17  ma      22584 non-null  object 
 18  pred    22584 non-null  int64  
dtypes: float64(16), int64(1), object(2)
memory usage: 3.3+ MB


In [ ]:
#Finding if there is any missing value
train_data.isna().sum()

pc         0
ld       222
m0         0
m1         0
m2         0
m3       314
m4       467
m5       576
m6       675
m7       788
m8       900
m9      1047
m10     1100
m11     1227
m12     1381
m13     1499
m14     1589
ma         0
pred       0
dtype: int64

In [ ]:
#Getting description of our numerical data
train_data.describe()

,ld,m0,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,m14,pred
count,22362.000000,22584.000000,22584.000000,22584.000000,22270.000000,22117.000000,22008.000000,21909.000000,21796.000000,21684.000000,21537.000000,21484.000000,21357.000000,21203.000000,21085.000000,20995.000000,22584.000000
mean,127.893905,78.002391,6.589621,65.664054,14.241585,123.207337,13.697683,183.894313,9.380175,199.786867,9.143454,169.239563,15.334764,122.868388,12.521853,168.530817,0.215772
std,39.078070,43.156539,3.360565,34.690224,0.801207,10.481427,0.798463,10.478648,0.800904,10.578785,0.799078,14.754047,1.214489,15.669803,0.923849,15.643776,0.411366
min,33.160000,0.000000,0.000000,0.000000,11.174400,84.084000,10.250400,133.507500,6.374400,159.778500,6.029600,100.170000,10.096000,54.253500,8.712000,95.970000,0.000000
25%,100.090000,42.000000,4.000000,42.000000,13.703200,116.067000,13.147200,176.809500,8.842200,192.651375,8.604000,159.715500,14.529600,112.365750,11.912000,158.109000,0.000000
50%,122.355000,73.500000,6.400000,63.000000,14.239200,123.207000,13.702400,183.949500,9.378800,199.741500,9.143200,169.359750,15.364000,123.007500,12.504000,168.441000,0.000000
75%,149.247500,105.000000,8.800000,84.000000,14.786400,130.357500,14.239200,190.932000,9.916000,206.944500,9.679200,178.741500,16.161600,133.465500,13.098400,179.361000,0.000000
max,385.860000,283.500000,23.200000,252.000000,17.199200,173.082000,17.140000,226.201500,12.335200,249.973500,12.329600,236.029500,20.512000,185.461500,18.170400,234.181500,1.000000


In [ ]:
#Make a correlation matrix
train_data.corr()

<ipython-input-17-edaffb053e21>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train_data.corr()


,ld,m0,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,m14,pred
ld,1.000000,-0.009238,0.000226,0.002705,-0.005950,-0.014638,-0.003876,0.007796,-0.003680,-0.004116,-0.010603,-0.008579,0.005206,-0.002975,0.006338,-0.001511,0.130629
m0,-0.009238,1.000000,-0.257880,-0.080624,-0.007035,-0.005844,0.004807,0.000388,0.002767,-0.009053,0.011413,0.032976,-0.001227,0.037249,-0.031348,0.024316,0.008733
m1,0.000226,-0.257880,1.000000,0.007679,-0.008784,-0.001143,-0.009745,0.004303,-0.017575,0.005140,0.005273,0.009872,-0.032612,-0.084544,0.004952,-0.009950,-0.008790
m2,0.002705,-0.080624,0.007679,1.000000,-0.002065,-0.012961,0.002776,-0.009204,0.002517,0.001086,0.005361,0.017842,-0.002497,-0.002552,0.031428,0.013817,0.018494
m3,-0.005950,-0.007035,-0.008784,-0.002065,1.000000,0.010305,0.009132,0.008121,0.006517,-0.002599,0.004569,-0.006351,-0.002585,0.002085,0.006421,0.005029,0.004113
m4,-0.014638,-0.005844,-0.001143,-0.012961,0.010305,1.000000,0.004450,0.006654,-0.005707,0.003638,0.009629,-0.010627,-0.006006,-0.003713,-0.003216,-0.000666,-0.012064
m5,-0.003876,0.004807,-0.009745,0.002776,0.009132,0.004450,1.000000,0.004948,-0.009843,-0.006564,-0.007988,0.005374,0.009484,0.000572,-0.012855,0.010186,0.018661
m6,0.007796,0.000388,0.004303,-0.009204,0.008121,0.006654,0.004948,1.000000,0.006150,-0.003976,0.002338,0.004286,-0.001526,-0.013345,-0.008908,0.001588,0.017465
m7,-0.003680,0.002767,-0.017575,0.002517,0.006517,-0.005707,-0.009843,0.006150,1.000000,0.004990,0.004859,0.004474,-0.008620,0.008540,-0.002244,0.003170,0.015771
m8,-0.004116,-0.009053,0.005140,0.001086,-0.002599,0.003638,-0.006564,-0.003976,0.004990,1.000000,0.002101,-0.004952,-0.015747,-0.005173,-0.005469,-0.008572,0.017072


In [ ]:
corr_matrix = train_data.corr()
fig, ax = plt.subplots(figsize = (15,10))
ax = sns.heatmap(corr_matrix,
                 annot = True,
                 linewidths = 0.5,
                 fmt = ".2f",
                 cmap = "YlGnBu"
             )

In [ ]:
#preprocessing
df_fin = train_data.copy()

In [ ]:
#Let us convert the object type data into categorical data

#This will turn all of the string values into categories
for label, content in df_fin.items():
    if pd.api.types.is_string_dtype(content):
        df_fin[label] = content.astype("category").cat.as_ordered()

In [ ]:
df_fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22584 entries, 0 to 22583
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   pc      22584 non-null  category
 1   ld      22362 non-null  float64 
 2   m0      22584 non-null  float64 
 3   m1      22584 non-null  float64 
 4   m2      22584 non-null  float64 
 5   m3      22270 non-null  float64 
 6   m4      22117 non-null  float64 
 7   m5      22008 non-null  float64 
 8   m6      21909 non-null  float64 
 9   m7      21796 non-null  float64 
 10  m8      21684 non-null  float64 
 11  m9      21537 non-null  float64 
 12  m10     21484 non-null  float64 
 13  m11     21357 non-null  float64 
 14  m12     21203 non-null  float64 
 15  m13     21085 non-null  float64 
 16  m14     20995 non-null  float64 
 17  ma      22584 non-null  category
 18  pred    22584 non-null  int64   
dtypes: category(2), float64(16), int64(1)
memory usage: 3.0 MB


In [ ]:
for label, content in df_fin.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)


ld
m3
m4
m5
m6
m7
m8
m9
m10
m11
m12
m13
m14


In [ ]:
#Fill numeric rows with median
for label, content in df_fin.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            #Add a binary column that tells if data was missing or not
            df_fin[label+"_is_missing"] = pd.isnull(content)
            #Fill missing numeric values with median
            df_fin[label] = content.fillna(content.median())

In [ ]:
# Turn categorical variables into numbers and fill missing
for label, content in df_fin.items():
    if not pd.api.types.is_numeric_dtype(content):
        #Add binary column to indicate whether sample had missing value
        df_fin[label + "_missing_"] = pd.isnull(content)
        #Turn categories into numbers and add +1
        df_fin[label] = pd.Categorical(content).codes + 1

In [ ]:
df_fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22584 entries, 0 to 22583
Data columns (total 34 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pc              22584 non-null  int8   
 1   ld              22584 non-null  float64
 2   m0              22584 non-null  float64
 3   m1              22584 non-null  float64
 4   m2              22584 non-null  float64
 5   m3              22584 non-null  float64
 6   m4              22584 non-null  float64
 7   m5              22584 non-null  float64
 8   m6              22584 non-null  float64
 9   m7              22584 non-null  float64
 10  m8              22584 non-null  float64
 11  m9              22584 non-null  float64
 12  m10             22584 non-null  float64
 13  m11             22584 non-null  float64
 14  m12             22584 non-null  float64
 15  m13             22584 non-null  float64
 16  m14             22584 non-null  float64
 17  ma              22584 non-null 

In [ ]:
df_fin.isna().sum()

pc                0
ld                0
m0                0
m1                0
m2                0
m3                0
m4                0
m5                0
m6                0
m7                0
m8                0
m9                0
m10               0
m11               0
m12               0
m13               0
m14               0
ma                0
pred              0
ld_is_missing     0
m3_is_missing     0
m4_is_missing     0
m5_is_missing     0
m6_is_missing     0
m7_is_missing     0
m8_is_missing     0
m9_is_missing     0
m10_is_missing    0
m11_is_missing    0
m12_is_missing    0
m13_is_missing    0
m14_is_missing    0
pc_missing_       0
ma_missing_       0
dtype: int64

In [ ]:
#model

In [ ]:
X = df_fin.drop("pred", axis  = 1)
y = df_fin["pred"]

In [ ]:
# splitting the data into train and validation sets
np.random.seed(42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

#Checking the shapes of these sets
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((18067, 33), (4517, 33), (18067,), (4517,))

In [ ]:
#Create a function to fit and score models

def fit_and_score(model, X_train, X_test, y_train, y_test):


    #Set random seed
    np.random.seed(42)

    #Fit the model to the data
    model.fit(X_train, y_train)

    #Making predictions
    y_pred = model.predict(X_val)
    #Evaluate the model and append its score to model_scores
    accuracy = model.score(X_test, y_test)
    f1 = f1_score(y_val, y_pred)

    return f1

In [ ]:
xgb_clf = xgb.XGBClassifier(
    n_estimators=100,
    random_state=42
)

In [ ]:
fit_and_score(xgb_clf, X_train, X_val, y_train, y_val)

0.06921675774134789

In [ ]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'reg_alpha': [0, 0.1, 0.5]
}

# Perform random search cross-validation
random_search = RandomizedSearchCV(
    estimator= xgb_clf,
    param_distributions=param_grid,
    n_iter=10,
    scoring='accuracy',
    cv=5,
    random_state=42,
    verbose = True
)

# Fit the random search to the training data
random_search.fit(X_train, y_train)




Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor=None, random_state=42, ...),
                   param_distributions={'colsample_bytree': [0.8, 1.0],
                                        'gamma': [0, 0.1, 0.2],
                                        'learning_rate': [0.1, 0.01, 0.001],
                                        'max_depth': [3, 5, 7],
                                        'n_estimators': [100, 200, 500],
                                        'reg_alpha': [0, 0.1, 0.5],
                                        'subsample': [0.8, 1.0]},
                   random_state=42, scoring='accuracy', verbose=True)

In [ ]:
# Get the best hyperparameters and score
best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Hyperparameters:", best_params)
print("Best Score:", best_score)

# Use the best model for prediction
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_val)

# Evaluate the f1_score of the best model
f1 = f1_score(y_val, y_pred)
print("F1 score:", f1)

Best Hyperparameters: {'subsample': 0.8, 'reg_alpha': 0.5, 'n_estimators': 300, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.8}
Best Score: 0.7829744890667975
F1 score: 0.0


In [ ]:
xgb_clf_best = xgb.XGBClassifier(
    n_estimators= 300,  subsample = 0.8
)

In [ ]:
xgb_clf.score(X_val, y_val)

0.773743635156077

In [ ]:
fit_and_score(xgb_clf_best,X_train, X_val, y_train, y_val)

0.1195219123505976

In [ ]:
test_data=pd.read_csv("/content/drive/MyDrive/capstone/Test_Data (1).csv")

In [ ]:
test_data

,index,pc,ld,m0,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,m14,ma
0,0,E,156.46,73.5,8.8,105.0,13.2768,113.5365,13.9552,168.5460,8.5896,187.6560,8.3472,178.5945,14.3192,138.0435,12.7008,166.1100,m76
1,1,y,58.87,84.0,12.8,42.0,14.4440,126.6930,13.6688,190.6275,10.1712,202.5870,8.5416,166.9815,15.5944,142.5690,11.6832,177.7230,m78
2,2,w,139.68,126.0,3.2,73.5,14.2432,107.4885,13.8544,168.3255,10.2504,214.2735,11.3392,155.1795,16.0256,132.1005,13.0104,185.6085,m78
3,3,z,194.06,52.5,6.4,105.0,14.5480,138.6105,13.5080,196.6545,10.6304,213.4650,9.9224,191.1000,14.8256,128.0475,12.1984,180.2745,m75
4,4,x,173.14,42.0,8.0,136.5,14.9496,103.8765,13.8936,168.3570,9.6184,191.2050,9.9400,185.0310,13.8664,124.7610,NaN,157.6470,m55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3981,3981,z,109.45,126.0,4.0,94.5,14.6168,131.4495,13.1032,189.0840,8.3384,203.2275,9.6848,NaN,14.6384,97.2510,12.3336,180.0435,m75
3982,3982,w,188.31,136.5,1.6,21.0,12.9032,115.1535,14.3616,189.4830,7.6680,183.8865,9.8032,172.2945,16.1512,110.7330,12.1680,204.5505,m78
3983,3983,E,80.99,31.5,4.8,115.5,15.0696,114.6810,14.0200,186.0810,9.2080,193.6200,9.1600,170.1840,NaN,127.4700,13.5016,NaN,m76
3984,3984,w,120.64,42.0,0.8,126.0,13.8960,123.6585,13.5496,187.8975,9.7944,189.0525,9.4752,173.0295,17.1728,113.0430,13.1480,164.2410,m78


In [ ]:

for label, content in test_data.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            #Add a binary column that tells if data was missing or not
            test_data[label+"_is_missing"] = pd.isnull(content)
            #Fill missing numeric values with median
            test_data[label] = content.fillna(content.median())

In [ ]:
for label, content in test_data.items():
    if pd.api.types.is_string_dtype(content):
        df_fin[label] = content.astype("category").cat.as_ordered()

In [ ]:
# Turn categorical variables into numbers and fill missing
for label, content in test_data.items():
    if not pd.api.types.is_numeric_dtype(content):
        #Add binary column to indicate whether sample had missing value
        test_data[label + "_missing_"] = pd.isnull(content)
        #Turn categories into numbers and add +1
        test_data[label] = pd.Categorical(content).codes + 1

In [ ]:
test_data.isna().sum()

index             0
pc                0
ld                0
m0                0
m1                0
m2                0
m3                0
m4                0
m5                0
m6                0
m7                0
m8                0
m9                0
m10               0
m11               0
m12               0
m13               0
m14               0
ma                0
ld_is_missing     0
m3_is_missing     0
m4_is_missing     0
m5_is_missing     0
m6_is_missing     0
m7_is_missing     0
m8_is_missing     0
m9_is_missing     0
m10_is_missing    0
m11_is_missing    0
m12_is_missing    0
m13_is_missing    0
m14_is_missing    0
pc_missing_       0
ma_missing_       0
dtype: int64

In [ ]:
xgb_clf_best = xgb.XGBClassifier(
    n_estimators= 300,  subsample = 0.8
)

In [ ]:
fit_and_score(xgb_clf_best,X_train, X_val, y_train, y_val)

0.1195219123505976

In [ ]:
test_data = test_data.drop("index", axis = 1)

In [ ]:
X_train.columns

Index(['pc', 'ld', 'm0', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9',
       'm10', 'm11', 'm12', 'm13', 'm14', 'ma', 'ld_is_missing',
       'm3_is_missing', 'm4_is_missing', 'm5_is_missing', 'm6_is_missing',
       'm7_is_missing', 'm8_is_missing', 'm9_is_missing', 'm10_is_missing',
       'm11_is_missing', 'm12_is_missing', 'm13_is_missing', 'm14_is_missing',
       'pc_missing_', 'ma_missing_'],
      dtype='object')

In [ ]:
y_pred_test = xgb_clf_best.predict(test_data)

In [ ]:
y_pred_test

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
predictions = pd.DataFrame()

In [ ]:
predictions["pred"] = y_pred_test

In [ ]:
predictions

,pred
0,0
1,0
2,0
3,1
4,0
...,...
3981,0
3982,0
3983,0
3984,0


In [ ]:
predictions.to_csv('/content/drive/MyDrive/capstone/Submission_SD.csv', index=False)